# Predictions

In this notebook you could test a pretrained model with real sentences

Links:
- https://developers.facebook.com/apps/285801978661570/webhooks/
- https://www.facebook.com/messages/t/intentbasedbot
- https://www.facebook.com/intentbasedbot/
- https://docs.google.com/presentation/d/1T9Yw5PtqbhyXggL7K05mxAH8ptI-_uXb8yCAv10jIfw/edit#slide=id.p    

In [1]:
%load_ext autoreload

In [2]:
%autoreload
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 
from intentbasedbot import model as intent_model

/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
company_name = 'cycles_motos'

In [7]:
company_utterances = {
    'cycles_motos': [
        'quiero saber cuanto me sale una moto financiada',
        'tienen llaves de paso de yamaha?',
        'necesito cambiar el tapon de aceite',        
        'quiero sacar una moto y que me descuenten por recibo de sueldo',
        'cuanto sale un casco',        
        'cuanto salen los cuatriciclos zanella para chicos',
        'soy monotributista, puedo sacar una honda?',
        'puedo pasar a buscar los engranajes de distribucion mañana',        
    ],
    'origenes': [
        'muy buenos dias quiero sacar un seguro de vida para mi mama y para mi',
        'quiero sacar una fe de vida',
        '¿donde tramito mi certificado de supervivencia?',
        'donde puedo dar de baja mi poliza',
        'quiero imprimir poliza de seguro'
    ],
    'smartfix': [
        'se me rompio la pc',
        'quiero formatear el disco',
        'se me cayo el iphone y se me rompio la pantalla', # Sacar iphone
        'necesito validar la garantia de mi HP',
        'tienen multifuncion HP?'
    ]        
}

In [8]:
for utterance in company_utterances[company_name]:
    print(intent_model.predict(utterance, company_name))

/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'utterance': 'quiero saber cuanto me sale una moto financiada', 'intent': 'cuotas', 'confidence': 98.59}
{'utterance': 'tienen llaves de paso de yamaha?', 'intent': 'repuestos', 'confidence': 97.68}


/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'utterance': 'necesito cambiar el tapon de aceite', 'intent': 'repuestos', 'confidence': 99.96}
{'utterance': 'quiero sacar una moto y que me descuenten por recibo de sueldo', 'intent': 'cuotas', 'confidence': 100.0}


/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'utterance': 'cuanto sale un casco', 'intent': 'repuestos', 'confidence': 39.04}
{'utterance': 'cuanto salen los cuatriciclos zanella para chicos', 'intent': 'consultasgenerales', 'confidence': 96.38}
{'utterance': 'soy monotributista, puedo sacar una honda?', 'intent': 'cuotas', 'confidence': 100.0}
{'utterance': 'puedo pasar a buscar los engranajes de distribucion mañana', 'intent': 'cuotas', 'confidence': 89.35}


/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ramiro/bin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
